# Cleaning the Household data
Before further analysis the data was cleaned and PII (Personally Indentifiable Information) removed or obscured.

In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
# load the data
households = pd.read_csv('data_raw_NOGIT/180613_households_district3_all_data.txt', sep='\t')
# and a cityarea look up file
cityareas = pd.read_csv('data_raw_NOGIT/cityareas_precinct.csv')

# loading the finalized look up table for Households 
# so I can re clean/tweak fix data issues without reshuffling
hid_lookup = pd.read_csv('data_clean/20180621_households_lookup_NO_GIT.csv', index_col='Hid')

For the Household data the value_counts() for each column were reviewed and the following was done to clean and anonymize the data:

| Original Data Column | Description of action | output column(s) | Type |
|:---:|:---|:---:|:---:|
| 'Household_Id' | Rows of table were randomly shuffled, the index reset and the new index used as new UID. | 'hid' | Num |
| 'FullAddress' | Shown to be concatenation of 'HouseNumber','Street','StreetType', 'BuildingNumber & 'ApartmentNumber' in all but 4 cases where the Apt numbers appeared to be missing or include typo's and then dropped as PII.  | | |
| 'HouseNumber' | Dropped as PII. | | |
| 'HouseNumberSuffix' | Dropped empty. | | |
| 'StreetPrefix' |  Dropped empty. | | |
| 'Street' | Used to clean 'StreetType' and then Dropped as PII. | | |
| 'StreetType' | ‘CMN’ ‘GREEN’ => ‘GRN’ and two cross streets => ‘UNK’. Also combined some smaller groups. | 'StreetType' | Cat |
| 'BuildingNumber' | Only 3 entries dropped. | | |
| 'ApartmentNumber' | Converted to a True/False field. | 'isApt' | Bool |
| 'City' | Dropped, all entries are the same. | | |
| 'State' | 6 missing rows, dropped as all should be the same. | | |
| 'Zip' | Cleaned all to 5 digit numerical zip code entries. | 'Zip' | Num |
| 'Precinct' | Converted to number and kept. | 'Precinct' | Num |
|  | The Precinct was also used to create a 'CityArea' column | 'CityArea' | Cat |
| 'PrecinctSub' | Converted to number and kept. | 'PrecinctSub' | Num |
| 'District' | Kept as is in case we need to add in other district data. | 'District' | Num |


In [3]:
h=households
# dropping cityarea so I can add it cleanly shortly
h = h.drop('CityArea', axis='columns')
print('Total number of households in data: {}'.format(len(h)))

Total number of households in data: 6930


### Cleaning StreetType
significant data is missing from this column, I used the 'Street' and 'Full address' to identify the missing categories 'GRN', 'UNK' and add them, I was also able to identify that COMMON and COMMONS had not been mapped correctly to 'CMN' and fixed that.

In [4]:
def get_full_address(row):
    
    build_num = row.BuildingNumber
    apt_num = row.ApartmentNumber
    if str(build_num) != 'nan':
        build_num = int(row.BuildingNumber)
        apt_num = '{}'.format(str(row.ApartmentNumber))
    elif str(row.ApartmentNumber) != 'nan':
        apt_num = '# {}'.format(str(row.ApartmentNumber))
        
    row['cc_full_add'] = ' '.join([x for x in [str(row.HouseNumber), row.Street,
                                 row.StreetType, str(build_num),
                                               apt_num] if str(x) != 'nan'])
    return row

In [5]:
h = h.apply(get_full_address, axis=1)

In [6]:
h[['FullAddress','cc_full_add']]
#assert (h.FullAddress == h.cc_full_add).all()
add_issues = h.loc[h['FullAddress'] != h['cc_full_add'], ['Household_Id','FullAddress','cc_full_add','BuildingNumber']]
print('All but {} address\'s follow the pattern.'.format(len(add_issues)))
#add_issues

All but 4 address's follow the pattern.


All other Voters living in WAINWRIGHT COMMON have no Apt address entered, the USPS doesn't recognize the Apt value T so set apt to NaN for that household. An address on BOSWELL TER has a typo in the apartment number - number is close to house number and the other voter in the household has the house number entered as the apt number.
The last two were clear typos so set to match the apt numbers included in the FullAddress.

In [7]:
# correcting these 4 entries 
h.iloc[[1467], 8] = np.nan  # to match USPS delivery address
h.iloc[[4458], 8] = np.nan  # to match USPS delivery address
h.iloc[[5790], 8] = 147 # to match household FullAddress
h.iloc[[5887], 8] = 337 # to match household FullAddress
#h.iloc[[1467,4458,5790,5887],8]

In [8]:
# Various views that helped understand the missing StreetType data.
# The Street names with null 'StreetType'
#h[h.StreetType.isnull()].loc[:,['Street', 'StreetType']].groupby('Street').count()
# most are 'COMMONS' in fact all Full Address' that have 'COMMON' in their name have StreetType = null
#h.loc[h.FullAddress.str.contains('COMMON'),['Street','StreetType']].groupby('Street').count()
# Address' that have 'CMN in StreetType
#h.loc[h.StreetType.str.contains('CMN').fillna(0) == True,['FullAddress', 'Street','StreetType']]

In [9]:
h.loc[:,'StreetTypeClean'] = h['StreetType']
# Cleaning the data by setting all Full Address' that include ' COMMON' to have StreetType 'CMN'
h.loc[h.FullAddress.str.contains(' COMMON') == True,['StreetTypeClean']] = 'CMN'
# Same for ' GREEN' => 'GRN', and the two cross street parital address' to 'UNK'
h.loc[h.FullAddress.str.contains(' GREEN') == True,['StreetTypeClean']] = 'GRN'
h.loc[h.FullAddress.str.contains('/') == True,['StreetTypeClean']] = 'UNK'

# combining some small categories toghether
for_other = ['RD', 'LN', 'PKWY', 'LOOP', 'GRN', 'CIR']
h.loc[h.StreetTypeClean.isin(['PL', 'TER']) == True,['StreetTypeClean']] = 'PL/TER'
h.loc[h.StreetTypeClean.isin(for_other) == True,['StreetTypeClean']] = 'OTH'

In [10]:
# check code
h.loc[(h.FullAddress.str.contains('COMMON') == True)
      |(h.FullAddress.str.contains(' GREEN') == True)
      |(h.FullAddress.str.contains('/') == True)
      ,['FullAddress', 'Street','StreetType', 'StreetTypeClean']]

t = h[['FullAddress', 'Street','StreetType', 'StreetTypeClean']].groupby('Street').count()
t.sum()

FullAddress        6930
StreetType         6097
StreetTypeClean    6930
dtype: int64

### Adding and cleaning CityArea
Using the Precinct information and a City Area look up table I was able to add a City Area to each HouseHold.

In [11]:
h = pd.merge(h, cityareas, on='Precinct', how='left')

In [12]:
h[['CityArea']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6930 entries, 0 to 6929
Data columns (total 1 columns):
CityArea    6815 non-null object
dtypes: object(1)
memory usage: 108.3+ KB


The initial look up table was missing two precincts, I identified the precincts.

In [13]:
h.loc[h.CityArea.isnull() == True,['Household_Id', 'Precinct']].groupby('Precinct').count()

,Household_Id
Precinct,
831730,111
835050,4


I identified the steets affected by the missing precincts:

In [14]:
PrecinctWithMissingCityArea = [831730, 835050]
h.loc[h['Precinct'].isin(PrecinctWithMissingCityArea), [
    'FullAddress','Street','CityArea','Precinct']].fillna('missing').groupby([
    'Precinct','CityArea','Street']).count()

FullAddress
Precinct CityArea Street                      
831730   missing  APPLETREE                  9
                  CHERRY BLOSSOM             9
                  FERNWOOD                   6
                  FREESIA                    9
                  FRUITWOOD                 12
                  JASMINE                   14
                  LAURUS                    11
                  PEACHTREE                 11
                  RIVERWALK                 15
                  ROSETREE                  12
                  SANDALWOOD                 3
835050   missing  CHAUNTRY COMMON            3
                  PERALTA                    1

The 831730 streets are all grouped in one fairly recent development around the Riverwalk road, using a map to see the other nearby streets I was able to identify the best `CityArea` for this `Precinct` is `Downtown / BART`

In [15]:
h.loc[(h.Street.str.contains('RIVERWALK'))
       | (h.Street.str.contains('TEMPLE'))
       | (h.Street.str.contains('BISHOP'))
       | (h.Street.str.contains('ARLENE'))
       | (h.Street.str.contains('GREENWOOD')),
      ['FullAddress', 'Street', 'CityArea', 'Precinct']].fillna('missing').groupby([
    'Precinct','CityArea','Street']).count()

,,,FullAddress
Precinct,CityArea,Street,
831730,missing,RIVERWALK,15
835040,Downtown / BART,BISHOP,55
835410,Downtown / BART,TEMPLE,11


Chauntry Common is another development where most of the houses are in one precinct and a few on the edge are in another, together with the corner house with address on Peralta Blvd. The best `CityArea` for this precinct is `Centerville`.

Note: Peralta is a very long road with residents voting in 5 different precincts, Chauntry Common is near the 3000 block of Peralta. 

In [16]:
h.loc[h.Street.str.contains('CHAUNTRY')
      | ((h.Street.str.contains('PERALTA')) & (h.HouseNumber > 3000))
      ,['HouseNumber','Street', 'CityArea', 'Precinct']].fillna('missing').sort_values('HouseNumber').groupby([
    'Precinct','CityArea','Street']).count()

HouseNumber
Precinct CityArea    Street                      
835010   Centerville CHAUNTRY COMMON           16
                     PERALTA                    2
835050   missing     CHAUNTRY COMMON            3
                     PERALTA                    1

In [17]:
h.loc[:,'CityAreaClean'] = h['CityArea']
# setting the CityArea for the 'missing' Precincts
h.loc[h.Precinct == 831730, 'CityAreaClean'] = 'Downtown / BART'
h.loc[h.Precinct == 835050, 'CityAreaClean'] = 'Centerville'

# validate
h.loc[h['Precinct'].isin(PrecinctWithMissingCityArea), [
    'FullAddress','Street','CityAreaClean','Precinct']].groupby([
    'Precinct','CityAreaClean','Street']).count()

FullAddress
Precinct CityAreaClean   Street                      
831730   Downtown / BART APPLETREE                  9
                         CHERRY BLOSSOM             9
                         FERNWOOD                   6
                         FREESIA                    9
                         FRUITWOOD                 12
                         JASMINE                   14
                         LAURUS                    11
                         PEACHTREE                 11
                         RIVERWALK                 15
                         ROSETREE                  12
                         SANDALWOOD                 3
835050   Centerville     CHAUNTRY COMMON            3
                         PERALTA                    1

### Cleaning the Zip data
Some zips had been entered with varying levels of accuracy. I cleaned them all to a consistent 5 digit zip code

In [18]:
h['ZipClean'] = h.Zip.astype(str).str[0:5].astype('int')
h['ZipClean'].value_counts()

94538    3835
94536    3095
Name: ZipClean, dtype: int64

### Creating an isApt field
A True/False column denoting if the household has an Apt number.

In [19]:
h['SameAptHouNo'] = h.apply(lambda row:'Y' if 
                                str(row.ApartmentNumber) == str(row.HouseNumber) else 'N', axis=1)
apt_issues = h.loc[(h.SameAptHouNo == 'Y')
          ,['Household_Id','ApartmentNumber', 'SameAptHouNo', ]]
print('All but {} Household\'s have different Apt # than House Numbers.'.format(len(apt_issues)))
#apt_issues

All but 3 Household's have different Apt # than House Numbers.


In [20]:
# correcting these 3 entries 
h.iloc[[1499], 8] = np.nan  # to match 3 other family members at same address
h.iloc[[1855], 8] = np.nan  # to match USPS delivery address
h.iloc[[6051], 8] = np.nan  # to match USPS delivery address
#h.iloc[[1499,1855,6051],[0,1,8]]

In [21]:
h['isApt'] = h.ApartmentNumber.notnull()
h.isApt.value_counts()

False    4927
True     2003
Name: isApt, dtype: int64

### Moving to Categories and renaming ready for output

In [22]:
h = h.loc[:,['Household_Id', 'StreetTypeClean', 'ZipClean', 'Precinct', 'PrecinctSub',
                 'District', 'CityAreaClean', 'isApt']]
h.rename(columns={'CityAreaClean':'CityArea','ZipClean':'Zip',
                      'StreetTypeClean':'StreetType'}, inplace=True)

#categories don't reload from .csv so will have to be reset after reading in data.
#h1[clean.select_dtypes(['object']).columns] = clean.select_dtypes(['object'
#                                                            ]).apply(lambda 
#                                                                     x: x.astype('category'))

### Shuffling the data to create new ID

In [23]:
# randomly shuffling the household row and reset index to make the new order the index
# commenting out so won't run again
#h = h.sample(frac=1).reset_index(drop=True)
#h.index.name = 'Hid'
#h = h.reset_index()

In [24]:
#hid_lookup = h[['Hid', 'Household_Id']]  # clean up of HH done after final shuffle complex

Working through recreating a cleaned data file ready for tweaks to the data cleaning

In [25]:
print(h.shape)
h = pd.merge(h, hid_lookup.reset_index(), on='Household_Id', how='left')

# ordering columns, sorting by Hid, removing Household_Id and resetting index 
# to make h match 'clean' data file
h = h[['Hid', 'StreetType', 'Zip', 'Precinct', 'PrecinctSub',
         'District', 'CityArea', 'isApt']].sort_values('Hid').reset_index(drop=True)
print(h.shape)

(6930, 8)
(6930, 8)


In [26]:
# loading prior 'clean' data for comparison
cl_hh = pd.read_csv('data_clean/20180621_households_district3.csv')

In [27]:
def find_changes(df1, df2):
    #Finding diffs in the data from first dump to second dump
    # https://stackoverflow.com/a/17095620/1215012

    ne_stacked = (df1 != df2).stack()
    changed = ne_stacked[ne_stacked]
    changed.index.names = ['id', 'col']

    difference_locations = np.where(df1 != df2)
    changed_from = df1.values[difference_locations]
    changed_to = df2.values[difference_locations]

    out = pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)
    out = out.reset_index()
    interesting = out.loc[(out['from'].notnull()) & (out['to'].notnull())]
    print(interesting.shape)
    print(interesting.col.unique())
    return interesting

In [28]:
cl_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6930 entries, 0 to 6929
Data columns (total 9 columns):
Hid            6930 non-null int64
StreetType     6930 non-null object
Zip            6930 non-null int64
Precinct       6930 non-null int64
PrecinctSub    6930 non-null int64
District       6930 non-null int64
CityArea       6930 non-null object
isApt          6930 non-null bool
cHid           6930 non-null int64
dtypes: bool(1), int64(6), object(2)
memory usage: 440.0+ KB


In [29]:
h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6930 entries, 0 to 6929
Data columns (total 8 columns):
Hid            6930 non-null int64
StreetType     6930 non-null object
Zip            6930 non-null int64
Precinct       6930 non-null int64
PrecinctSub    6930 non-null int64
District       6930 non-null int64
CityArea       6930 non-null object
isApt          6930 non-null bool
dtypes: bool(1), int64(5), object(2)
memory usage: 385.8+ KB


In [30]:
# Validating that only the items changed in this version of the notebook have actually changed
a = cl_hh.drop(['StreetType','cHid'], axis='columns')
b = h.drop('StreetType', axis='columns')
find_changes(a, b)
a.equals(b)

(0, 4)
[]


True

## Household clean up
3 additional households are in the data due to house numbers being entered as apartment numbers. Creating a cleaned Hid that corrects for this.

In [31]:
hid_lookup['cHid'] = hid_lookup.index

In [32]:
# Used Household_Id to lookup correct index' for 6 affected households

# combining these 6 into 3 cHid 
hid_lookup.iloc[[4849], 1] = h.iloc[[489]].index  # setting 4849's cHid to 489's Hid
hid_lookup.iloc[[6399], 1] = h.iloc[[478]].index  # setting 6399's cHid to 478's Hid
hid_lookup.iloc[[6208], 1] = h.iloc[[2953]].index # setting 6208's cHid to 2953's Hid
#hid_lookup.iloc[[489,4849,6399,478,6208,2953]]

In [33]:
# saving cleaned file out
#date = pd.Timestamp("today").strftime("%Y%m%d")
#hid_lookup.to_csv('data_clean/{}_households_lookup_NO_GIT.csv'.format(date))

In [34]:
# check code used when first creating cHid
#find_changes(hid_lookup[['Household_Id']], hid_lookup_old)
#hid_lookup[['Household_Id']].equals(hid_lookup_old)

In [35]:
# Adding new cHid column to h for output 
h = pd.merge(h, hid_lookup[['cHid']].reset_index(), on='Hid', how='left')

In [36]:
# check code
#h.iloc[4847:4852]

### Creating the output clean files

In [37]:
clean = h.copy()
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6930 entries, 0 to 6929
Data columns (total 9 columns):
Hid            6930 non-null int64
StreetType     6930 non-null object
Zip            6930 non-null int64
Precinct       6930 non-null int64
PrecinctSub    6930 non-null int64
District       6930 non-null int64
CityArea       6930 non-null object
isApt          6930 non-null bool
cHid           6930 non-null int64
dtypes: bool(1), int64(6), object(2)
memory usage: 494.0+ KB


In [38]:
clean.set_index('Hid', inplace=True)
# hid_lookup.set_index('Hid', inplace=True) # using existing lookup

In [39]:
date = pd.Timestamp("today").strftime("%Y%m%d")
clean.to_csv('data_clean/{}_households_district3.csv'.format(date))
# hid_lookup.to_csv('data_clean/{}_households_lookup_NO_GIT.csv'.format(date))

### Change Management
Initial clean data and hid_lookup created 20180616

20180621:
- corrected ApartmentNumber for 2 address' with full address pattern problems
- corrected ApartmentNumber for 3 households were the Apt number matched the House number
- additional cHid column added to hid_lookup to combine the 3 duplicated HH's

20180627:
- StreetType, combined smaller groups
- standardized on UNK for unknown to match existing coding used for Party